# create table

In [ ]:
import pymysql as mariadb

try:
    conn = mariadb.connect(
     host='db',
     user='root',
     passwd='1234',
     db='rental',
     charset='utf8mb4'
    )
    
    cursor = conn.cursor()
    sql = """
        CREATE TABLE `rentInfo` (
  `ID` INT NOT NULL AUTO_INCREMENT,
  `url` VARCHAR(100) NULL,
  `update_date` VARCHAR(50) NULL,
  `title` VARCHAR(100) NULL,
  `address` VARCHAR(200) NULL,
  `label` CHAR(5) NULL,
  `temp` CHAR(1) NULL,
  `sex` CHAR(1) NULL,
  `smoke` CHAR(1) NULL,
  `pet` CHAR(1) NULL,
  `cook` CHAR(1) NULL,
  `space` DECIMAL(9,2) NULL,
  `rent` INT NULL,
  `floor` INT NULL,
  `stories` INT NULL,
  `pattern` VARCHAR(20) NULL,
  `landlord` VARCHAR(100) NULL,
  `description` TEXT(2000) NULL,
  `lat` DECIMAL(20,17) NULL,
  `lng` DECIMAL(20,17) NULL,
  `cityID` INT NULL,
  PRIMARY KEY (`ID`),
  UNIQUE INDEX `url_UNIQUE` (`url` ASC))
  ENGINE = InnoDB DEFAULT  
  CHARACTER SET = utf8mb4 COLLATE = utf8mb4_general_ci;
        """
    try:
        cursor.execute(sql)
    except Exception as e:
        print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
        cursor.close()
        conn.close()
except Exception as e:
    print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

# read all json & drop into Queue

In [ ]:
import glob
from multiprocessing import Process, Queue
import json
q1=Queue()
fil = glob.glob('rent_data/*')
for filLis in fil:
    for detail in glob.glob('{}/*'.format(filLis)):
        with open('{}'.format(detail),encoding='utf-8') as f:
            jSon = json.load(f)
            #print(jSon)
            q1.put(jSon)

print(q1.qsize())

# write method

In [ ]:
def write(lock,obj):
    obj = obj
    lock.acquire()
    try:
        conn = mariadb.connect(
            host='db',
            user='root',
            passwd='1234',
            db='rental',
            charset='utf8mb4'
        )
        cursor = conn.cursor()
        sql = '''INSERT INTO rentInfo (url,update_date,title,address,label,temp,sex,pet,cook,smoke,space,floor,stories,
                pattern,rent,landlord,description,lat,lng,cityID) VALUES 
                (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
        values = (obj['url'], obj['update'],obj['title'], obj['address'],obj['label'], obj['temp'],
                obj['sex'], obj['pet'], obj['cook'], obj['smoke'],obj['space'], obj['floor'], obj['stories'],
                obj['pattern'], obj['rent'],obj['landlord'], obj['description'], obj['lat'],obj['lng'], obj['cityID'])
        try:
            cursor.execute(sql,values)
            conn.commit()
        except Exception as e:
            print(obj)
            conn.rollback()
            print('Execute Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))

        cursor.close()

        conn.close()
    except Exception as e:
        print(obj)
        print('Connection Failed!\nError Code is %s;\nError Content is %s;' % (e.args[0],e.args[1]))
    finally:
        lock.release()

# run

In [ ]:
import pymysql as mariadb
import time
from multiprocessing import Process, Lock,
lock = Lock()
start = time.time()
while not q1.empty():
    try:
        obj = q1.get()
        p = Process(target=write, args=(lock, obj)) 
        p.start()
        p.join(timeout=0.1)
    except Exception as e:
        print(e)
        print(obj)
print('total time = {}'.format(time.time()-start))